# 411 - Uphill Paths

## Problem Statement

Let $n$ be a positive integer. Suppose there are stations at the coordinates $(x, y) = (2^i \bmod n, 3^i \bmod n)$ for $0 \leq i \leq 2n$. We will consider stations with the same coordinates as the same station.

We wish to form a path from $(0, 0)$ to $(n, n)$ such that the $x$ and $y$ coordinates never decrease.<br>
Let $S(n)$ be the maximum number of stations such a path can pass through.

For example, if $n = 22$, there are $11$ distinct stations, and a valid path can pass through at most $5$ stations. Therefore, $S(22) = 5$.

The case is illustrated below, with an example of an optimal path:

<div class="center">
    <img src="images/0411_longpath.png" alt="png411.png">
</div>

It can also be verified that $S(123) = 14$ and $S(10000) = 48$.

Find $\sum S(k^5)$ for $1 \leq k \leq 30$.


## Solution

There are two challenges in this problem. The first one is to generate all the points. The implementation below does not make any attempt to exploit patterns in the station generation so this is the main bottleneck. Here we simply generate the points and add them to a set to deal with the repeated stations. In order to speed up the generation, we use a numba compiled functions (including for exponentiation by squaring). The second challenge is to find the maximum number of stations that can be visited. This can be solved using a 2-dimensional version of the classical [Longest Increasing Subsequence (LIS)](https://leetcode.com/problems/longest-increasing-subsequence/editorial/) problem. We first sort the points based on the $x$-values then $y$-values and apply LIS on the $y$-values. Efficient LIS implementation is $O(n \log n)$.

One thing that requires special attention is that there should not be any station at $(0, 0)$. However, when setting $k = 1$, on such station is created. Therefore, we need to subtract one from the total.

In [2]:
import bisect
from numba import njit

@njit
def modular_exponentiation(base, exponent, mod):
    result = 1
    current_base = base % mod  # Reduce base modulo first to keep numbers small

    while exponent > 0:
        if exponent % 2 == 1:  # If exponent is odd, multiply the current result by the base
            result = (result * current_base) % mod
        current_base = (current_base * current_base) % mod  # Square the base
        exponent //= 2  # Divide the exponent by 2

    return result


@njit
def generate_stations(n):
    stations = set()
    for i in range(2 * n + 1):
        stations.add((modular_exponentiation(2, i, n), modular_exponentiation(3, i, n)))
    return stations


def lis(nums):
    sub = []
    for num in nums:
        i = bisect.bisect_right(sub, num)
        if i == len(sub):
            sub.append(num)
        else:
            sub[i] = num
    return len(sub)

In [3]:
res = -1  ## Subtract 1 to account for the (0, 0) station generated when k=1
for k in range(1, 31):
    stations = generate_stations(k**5)
    stations = sorted(stations)
    ys = [stations[i][1] for i in range(len(stations))]
    res += lis(ys)